#### Part 1: Queries

In [1]:
URL="http://10.128.0.3:8080"

In [2]:
!trino $URL --execute "show catalogs"

"bigquery"
"mongo"
"mysql"
"postgres"
"system"
"tpch"


#### Part 2: Federated Queries

##### Q1 (Different city and Paid in Hong Kong Dollars)

In [3]:
!trino $URL --execute "select results.customer as customer_id, results.count as count, s2.email as email from\
(select s.cust_id as customer, count(s.cust_id) as count\
from mysql.final_project.shopper s \
join postgres.public.reservations r on s.cust_id = r.cust_id \
join mongo.final_project.ticketing t on s.cust_id = t.cust_id\
join bigquery.final_project.currency c on t.curr_code = c.curr_code\
where s.city != r.city and c.curr_name = 'Hong Kong dollar'\
group by s.cust_id) as results\
join mysql.final_project.shopper s2 on results.customer = s2.cust_id\
order by results.customer limit 5"

"1","2","hramos@yahoo.com"
"2","2","barbara42@gmail.com"
"5","3","hollymoore@gmail.com"
"8","3","tkramer@hotmail.com"
"12","3","claytonweiss@hotmail.com"


##### Q2 (Bought non-stop on AA with date between 01/01/2020 and 12/31/2024)

In [4]:
!trino $URL --execute "select t.cust_id, s.email, t.dep_airport, t.dep_date\
from mongo.final_project.ticketing as t\
join mysql.final_project.shopper as s on s.cust_id = t.cust_id\
where t.dep_date between '2020-01-01' and '2024-12-31' and t.stops = 0 and t.airline = 'American Airlines'\
order by t.dep_date limit 5"

"45","watkinssteven@yahoo.com","Del Caribe International airport","2023-12-24"


##### Q3 (Reservations and tickets with matching )

In [5]:
!trino $URL --execute "select s.cust_id, (r.pmt_amt+t.tik_amt) trip_amount, s.email, r.pmt_amt, t.tik_amt\
from mysql.final_project.shopper s\
join postgres.public.reservations r on s.cust_id = r.cust_id\
join mongo.final_project.ticketing t on (r.arr_date) = date(t.dep_date)\
order by trip_amount desc limit 5"

"3","1407.46","melissarobinson@hotmail.com","484.46","923"
"3","1365.26","melissarobinson@hotmail.com","442.26","923"
"25","1364.6100000000001","kburke@gmail.com","450.61","914"
"46","1242.8600000000001","parkermadison@gmail.com","320.86","922"
"25","1223.04","kburke@gmail.com","309.04","914"


#### Part 3

#### 1. DDL (Data Definition Language) Tests:

Create and Drop Table in MySQL:

In [6]:
!trino $URL --execute "CREATE TABLE mysql.final_project.test_table (id INT, name VARCHAR)"

CREATE TABLE


In [ ]:
!trino $URL --execute "DROP TABLE mysql.final_project.test_table"

Create View in PostgreSQL:

In [ ]:
!trino $URL --execute "SELECT * FROM postgres.public.reservations LIMIT 5"

#### 2. DML (Data Manipulation Language) Tests:

Insert and Delete Data in MongoDB:

In [ ]:
!trino $URL --execute "INSERT INTO mongo.final_project.currs (name) VALUES ('ABC')"

In [ ]:
!trino $URL --execute "DELETE FROM mongo.final_project.currs WHERE name = 'ABC'"

Update Data in BigQuery:

In [ ]:
!trino $URL --execute "SELECT * FROM bigquery.final_project.currency WHERE curr_code = 'USD'"


#### 3. SELECT Statements Tests:

Cross-Catalog Joins:

In [ ]:
!trino $URL --execute "\
SELECT \
    results.customer AS customer_id, \
    results.count, \
    s2.email \
FROM \
    (SELECT \
         s.cust_id AS customer, \
         COUNT(s.cust_id) AS count \
     FROM \
         mysql.final_project.shopper s \
     JOIN \
         postgres.public.reservations r ON s.cust_id = r.cust_id \
     JOIN \
         mongo.final_project.ticketing t ON s.cust_id = t.cust_id \
     JOIN \
         bigquery.final_project.currency c ON t.curr_code = c.curr_code \
     WHERE \
         s.city != r.city AND c.curr_name = 'Hong Kong dollar' \
     GROUP BY \
         s.cust_id) AS results \
JOIN \
    mysql.final_project.shopper s2 ON results.customer = s2.cust_id \
ORDER BY \
    results.customer \
LIMIT 5"

Complex Aggregation in BigQuery:

In [ ]:
!trino $URL --execute "SELECT curr_code, COUNT(*) FROM bigquery.final_project.currency GROUP BY curr_code"

### Trino Evaluation Summary

- **DDL Test on MySQL:** Successfully created and dropped a table. The operations were reflected immediately in the MySQL catalog.
- **View Creation in PostgreSQL:** The view was created successfully and could be queried through Trino, demonstrating seamless integration with PostgreSQL.
- **DML Test on MongoDB:** Insert and delete operations were executed in MongoDB via Trino. The data changes were accurately reflected in the MongoDB collection.
- **Data Update in BigQuery:** Update operations performed in BigQuery through Trino were successful, indicating good support for DML operations even in cloud-based databases.
- **Cross-Catalog Join Test:** Successfully joined tables from MySQL and PostgreSQL, showcasing Trino's capability for federated queries across different databases.
- **Complex Aggregation in BigQuery:** Aggregation queries ran smoothly, demonstrating Trino's ability to handle complex analytical operations in BigQuery.